<a href="https://colab.research.google.com/github/riadYoork/IMLO_exam/blob/main/IMLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

# Define image dimension
im_dimension = 224

# Define data transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((im_dimension, im_dimension)),
        transforms.CenterCrop(im_dimension),
        transforms.RandomRotation(15),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize((im_dimension, im_dimension)),
        transforms.CenterCrop(im_dimension),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((im_dimension, im_dimension)),
        transforms.CenterCrop(im_dimension),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

# Define datasets
data_dir = './data'
train_dataset = datasets.Flowers102(root=data_dir, split='train', download=True, transform=data_transforms['train'])
valid_dataset = datasets.Flowers102(root=data_dir, split='val', download=True, transform=data_transforms['valid'])
test_dataset = datasets.Flowers102(root=data_dir, split='test', download=True, transform=data_transforms['test'])

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)


100%|██████████| 344862509/344862509 [00:23<00:00, 14482513.39it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 743219.42it/s]


100%|██████████| 14989/14989 [00:00<00:00, 18345031.41it/s]


In [3]:

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class AdvancedFlowerNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.Linear1 = nn.Linear(3456, 1728)
        self.Linear2 = nn.Linear(1728, 102)
        self.Dropout = nn.Dropout(0.07)
        self.Conv1 = nn.Conv2d(3, 128, 5, 2)
        self.BNorm1 = nn.BatchNorm2d(128)
        self.Conv2 = nn.Conv2d(128, 256, 5, 2)
        self.BNorm2 = nn.BatchNorm2d(256)
        self.Conv3 = nn.Conv2d(256, 384, 5, 1)
        self.BNorm3 = nn.BatchNorm2d(384)
        self.Conv4 = nn.Conv2d(384, 384, 5, 1)
        self.BNorm4 = nn.BatchNorm2d(384)
        self.Conv5 = nn.Conv2d(384, 384, 5, 1)
        self.BNorm5 = nn.BatchNorm2d(384)
        self.Pool = nn.MaxPool2d((2, 2), (2, 2))
        self.ReLU = nn.ReLU()

    def forward(self, x):
        x = self.ReLU(self.BNorm1(self.Conv1(x)))
        x = self.Pool(x)
        x = self.ReLU(self.BNorm2(self.Conv2(x)))
        x = self.Dropout(x)
        x = self.ReLU(self.BNorm3(self.Conv3(x)))
        x = self.Pool(x)
        x = self.ReLU(self.BNorm4(self.Conv4(x)))
        x = self.Dropout(x)
        x = self.ReLU(self.BNorm5(self.Conv5(x)))
        x = x.reshape(-1, 3456)
        x = self.ReLU(self.Linear1(x))
        x = self.Linear2(x)
        return x



In [4]:
# Initialize the model
model = AdvancedFlowerNet().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.01)

# Add the learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.9)


In [5]:
import torch
from tqdm import tqdm



best_val_accuracy = 0
patience_counter = 0
early_stopping_patience = 10

def evaluate(loader):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    avg_loss = running_loss / len(loader)
    accuracy = 100 * correct / total
    return avg_loss, accuracy

def train_loop(epochs):
    global best_val_accuracy, patience_counter
    for epoch in range(epochs):
        model.train()
        running_loss = 0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        val_loss, val_accuracy = evaluate(valid_loader)
        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {running_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

        scheduler.step(val_loss)

        # Save the best model in sample_data directory
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), '/content/sample_data/best_model.pth')
            print(f'Saved Best Model with Validation Accuracy: {best_val_accuracy:.2f}%')
            patience_counter = 0
        else:
            patience_counter += 1


def test_loop():
    model.load_state_dict(torch.load('/content/sample_data/best_model.pth'))
    model.to(device)
    test_loss, test_accuracy = evaluate(test_loader)
    print(f'Final Test Loss: {test_loss:.4f}, Final Test Accuracy: {test_accuracy:.2f}%')

train_loop(600)
test_loop()


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 1/600, Train Loss: 4.3859, Val Loss: 3.9151, Val Accuracy: 9.31%
Saved Best Model with Validation Accuracy: 9.31%


Epoch 2/600, Train Loss: 3.6481, Val Loss: 3.4601, Val Accuracy: 16.18%
Saved Best Model with Validation Accuracy: 16.18%


Epoch 3/600, Train Loss: 3.1292, Val Loss: 3.3214, Val Accuracy: 17.25%
Saved Best Model with Validation Accuracy: 17.25%


Epoch 4/600, Train Loss: 2.7721, Val Loss: 3.1716, Val Accuracy: 20.78%
Saved Best Model with Validation Accuracy: 20.78%


...

Epoch 596/600, Train Loss: 0.0931, Val Loss: 1.6803, Val Accuracy: 58.92%


Epoch 597/600, Train Loss: 0.0638, Val Loss: 1.6018, Val Accuracy: 61.08%


Epoch 598/600, Train Loss: 0.0626, Val Loss: 1.5802, Val Accuracy: 62.35%


Epoch 599/600, Train Loss: 0.0601, Val Loss: 1.5821, Val Accuracy: 63.53%


Epoch 600/600, Train Loss: 0.0628, Val Loss: 1.5701, Val Accuracy: 63.43%
Final Test Loss: 1.7609, Final Test Accuracy: 60.40%
